In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
houses = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
print(houses.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [3]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# IMPORT DATA

In [4]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

# FILL THE MISSING VALUES

In [5]:
def fill_all_missing_values(data):
    data['FireplaceQu'].fillna("No", inplace=True)
    data['BsmtQual'].fillna("No", inplace=True)
    data['BsmtCond'].fillna("No", inplace=True)
    data['BsmtFinType1'].fillna("No", inplace=True)
    data['BsmtFinType2'].fillna("No", inplace=True)
    data['BsmtFinType2'].fillna("None", inplace=True)
    data['GarageType'].fillna("NA", inplace=True)
    for col in data.columns:
        if((data[col].dtype == 'float64') or (data[col].dtype == 'int64')):
            data[col].fillna(data[col].mean(), inplace=True)
        else:
             data[col].fillna(data[col].mode()[0], inplace=True)


fill_all_missing_values(train)
fill_all_missing_values(test)

# PUTTING TRANSFORMATIONS INTO FUNCTION

In [6]:
def recode_vars(data):
    data["MSSubClass_Recode"] = "Cheap"
    data.loc[data["MSSubClass"] == 20, "MSSubClass_Recode"] = "Normal"
    data.loc[data["MSSubClass"] == 150, "MSSubClass_Recode"] = "Normal"
    data.loc[data["MSSubClass"] == 60, "MSSubClass_Recode"] = "Expensive"
    data.loc[data["MSSubClass"] == 120, "MSSubClass_Recode"] = "Expensive"

    data["MSZoning_Recode"] = "Cheap"
    data.loc[data["MSZoning"] == "RL", "MSZoning_Recode"] = "Normal"
    data.loc[data["MSZoning"] == "FV", "MSZoning_Recode"] = "Expensive"

    data["LotShape_Recode"] = "Expensive"
    data.loc[data["LotShape"] == "Reg", "LotShape_Recode"] = "Normal"

    data["LandContour_Recode"] = "Expensive"
    data.loc[data["LandContour"] == "Bnk", "LandContour_Recode"] = "Cheap"
    data.loc[data["LandContour"] == "Lvl", "LandContour_Recode"] = "Normal"

    data["LotConfig_Recode"] = "Normal"
    data.loc[data["LotConfig"] == "Inside", "LotConfig_Recode"] = "Cheap"
    data.loc[data["LotConfig"] == "CulDSac", "LotConfig_Recode"] = "Expensive"

    data["LandSlope_Recode"] = "Expensive"
    data.loc[data["LandSlope"] == "Gtl", "LandSlope_Recode"] = "Normal"

    cheap_neighborhood = ["Blueste", "BrDale", "BrkSide", "Edwards", "IDOTRR", "MeadowV", "Mitchel", "NAmes", "NPkVill",
                         "OldTown", "SWISU", "Sawyer"]
    medium_neighborhood = ["Blmngtn", "CollgCr", "Crawfor", "Gilbert", "NWAmes", "SawyerW"]
    expensive_neighborhood = ["ClearCr", "NoRidge", "NridgHt", "Somerst", "StoneBr", "Timber", "Veenker"]

    data["Neighborhood_Recode"] = "Unassigned"

    for n in cheap_neighborhood:
        data.loc[data["Neighborhood"] == n, "Neighborhood_Recode"] = "Cheap"
    for n in medium_neighborhood:
        data.loc[data["Neighborhood"] == n, "Neighborhood_Recode"] = "Medium"
    for n in expensive_neighborhood:
        data.loc[data["Neighborhood"] == n, "Neighborhood_Recode"] = "Expensive"

    data["HouseStyle_Recode"] = "Cheap"
    data.loc[data["HouseStyle"] == "2Story", "HouseStyle_Recode"] = "Expensive"
    data.loc[data["HouseStyle"] == "2.5Fin", "HouseStyle_Recode"] = "Expensive"

    data["Exterior1st_Recode"] = "Cheap"
    data.loc[data["Exterior1st"] == "CemntBd", "Exterior1st_Recode"] = "Expensive"
    data.loc[data["Exterior1st"] == "VinylSd", "Exterior1st_Recode"] = "Expensive"
    data.loc[data["Exterior1st"] == "Stone", "Exterior1st_Recode"] = "Expensive"

    data["MasVnrType_Recode"] = "Cheap"
    data.loc[data["MasVnrType"] == "BrkFace", "MasVnrType_Recode"] = "Normal"
    data.loc[data["MasVnrType"] == "Stone", "MasVnrType_Recode"] = "Expensive"

    data["ExterQual_Recode"] = "Cheap"
    data.loc[data["ExterQual"] == "Gd", "ExterQual_Recode"] = "Expensive"
    data.loc[data["ExterQual"] == "Ex", "ExterQual_Recode"] = "Expensive"

    data["Foundation_Recode"] = "Cheap"
    data.loc[data["Foundation"] == "PConc", "Foundation_Recode"] = "Expensive"

    data["BsmtQual_Recode"] = "Cheap"
    data.loc[data["BsmtQual"] == "Gd", "BsmtQual_Recode"] = "Normal"
    data.loc[data["BsmtQual"] == "Ex", "BsmtQual_Recode"] = "Expensive"

    data["BsmtExposure_Recode"] = "Normal"
    data.loc[data["BsmtExposure"] == "No", "BsmtExposure_Recode"] = "Cheap"
    data.loc[data["BsmtExposure"] == "Gd", "BsmtExposure_Recode"] = "Expensive"

    data["BsmtFinType1_Recode"] = "Cheap"
    data.loc[data["BsmtFinType1"] == "GLQ", "BsmtFinType1_Recode"] = "Expensive"

    data["HeatingQC_Recode"] = "Cheap"
    data.loc[data["HeatingQC"] == "Gd", "HeatingQC_Recode"] = "Normal"
    data.loc[data["HeatingQC"] == "Ex", "HeatingQC_Recode"] = "Expensive"

    data["Electrical_Recode"] = "Cheap"
    data.loc[data["Electrical"] == "SBrkr", "Electrical_Recode"] = "Normal"

    data["KitchenQual_Recode"] = "Normal"
    data.loc[data["KitchenQual"] == "Gd", "KitchenQual_Recode"] = "Expensive"
    data.loc[data["KitchenQual"] == "Ex", "KitchenQual_Recode"] = "Very expensive"

    data["GarageType_Recode"] = "Cheap"
    data.loc[data["GarageType"] == "Attchd", "GarageType_Recode"] = "Normal"
    data.loc[data["GarageType"] == "BuiltIn", "GarageType_Recode"] = "Expensive"

    from sklearn.preprocessing import OrdinalEncoder

    ordinal_col = ["MSSubClass_Recode", "MSZoning_Recode", "LotShape_Recode", "LandContour_Recode", "LotConfig_Recode",
                  "Neighborhood_Recode", "HouseStyle_Recode", "Exterior1st_Recode", "MasVnrType_Recode",
                  "ExterQual_Recode", "Foundation_Recode", "BsmtQual_Recode", "BsmtExposure_Recode", 
                  "BsmtFinType1_Recode", "HeatingQC_Recode", "CentralAir", "Electrical_Recode", "KitchenQual_Recode", "FirePlaceQu",
                  "GarageType_Recode", "GarageFinish"]

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["MSSubClass_Recode"] = OE.fit_transform(data[["MSSubClass_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["MSZoning_Recode"] = OE.fit_transform(data[["MSZoning_Recode"]])

    OE = OrdinalEncoder(categories=[['Normal', 'Expensive']])
    data["LotShape_Recode"] = OE.fit_transform(data[["LotShape_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["LandContour_Recode"] = OE.fit_transform(data[["LandContour_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["LotConfig_Recode"] = OE.fit_transform(data[["LotConfig_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Medium', 'Expensive']])
    data["Neighborhood_Recode"] = OE.fit_transform(data[["Neighborhood_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Expensive']])
    data["HouseStyle_Recode"] = OE.fit_transform(data[["HouseStyle_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Expensive']])
    data["Exterior1st_Recode"] = OE.fit_transform(data[["Exterior1st_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["MasVnrType_Recode"] = OE.fit_transform(data[["MasVnrType_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Expensive']])
    data["ExterQual_Recode"] = OE.fit_transform(data[["ExterQual_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Expensive']])
    data["Foundation_Recode"] = OE.fit_transform(data[["Foundation_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["BsmtQual_Recode"] = OE.fit_transform(data[["BsmtQual_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["BsmtExposure_Recode"] = OE.fit_transform(data[["BsmtExposure_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Expensive']])
    data["BsmtFinType1_Recode"] = OE.fit_transform(data[["BsmtFinType1_Recode"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["HeatingQC_Recode"] = OE.fit_transform(data[["HeatingQC_Recode"]])

    OE = OrdinalEncoder(categories=[['N', 'Y']])
    data["CentralAir"] = OE.fit_transform(data[["CentralAir"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal']])
    data["Electrical_Recode"] = OE.fit_transform(data[["Electrical_Recode"]])

    OE = OrdinalEncoder(categories=[['Normal', 'Expensive', 'Very expensive']])
    data["KitchenQual_Recode"] = OE.fit_transform(data[["KitchenQual_Recode"]])

    OE = OrdinalEncoder(categories=[['No', 'Po', 'Fa', 'TA', 'Gd', 'Ex']])
    data["FireplaceQu"] = OE.fit_transform(data[["FireplaceQu"]])

    OE = OrdinalEncoder(categories=[['Cheap', 'Normal', 'Expensive']])
    data["GarageType_Recode"] = OE.fit_transform(data[["GarageType_Recode"]])

    OE = OrdinalEncoder(categories=[['Unf', 'RFn', 'Fin']])
    data["GarageFinish"] = OE.fit_transform(data[["GarageFinish"]])

# CALLING THE FUNCTION TO OUR DATA

In [7]:
recode_vars(train)
recode_vars(test)

from sklearn.preprocessing import StandardScaler
def df_preproc_columns(table):
    numeric_columns = ["LotFrontage", "LotArea", "YearBuilt", "YearRemodAdd", "MasVnrArea",
                      "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "GrLivArea",
                      "GarageYrBlt", "GarageArea", "WoodDeckSF", "OpenPorchSF", "3SsnPorch", "ScreenPorch"]
    table[numeric_columns] = table[numeric_columns].fillna(table[numeric_columns].mean())
    scaler = StandardScaler()
    table[numeric_columns] = scaler.fit_transform(table[numeric_columns])
    return table

train = df_preproc_columns(train)
test = df_preproc_columns(test)

# SELECTING RELEVANT VARIABLES

In [8]:
vars_to_keep = ["MSSubClass_Recode", "MSZoning_Recode", "LotFrontage", "LotArea", "LotShape_Recode",
               "LandContour_Recode", "LotConfig_Recode", "Neighborhood_Recode", "HouseStyle_Recode",
               "OverallQual", "YearBuilt", "YearRemodAdd", "Exterior1st_Recode", "MasVnrType_Recode",
               "MasVnrArea", "ExterQual_Recode", "Foundation_Recode", "BsmtQual_Recode",
               "BsmtExposure_Recode", "BsmtFinType1_Recode", "BsmtFinSF1", "BsmtUnfSF", 
               "TotalBsmtSF", "HeatingQC_Recode", "CentralAir", "Electrical_Recode",
               "1stFlrSF", "2ndFlrSF", "GrLivArea", "BsmtFullBath", "FullBath", "HalfBath",
               "BedroomAbvGr", "KitchenQual_Recode", "TotRmsAbvGrd", "Fireplaces", "FireplaceQu",
               "GarageType_Recode", "GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea",
               "WoodDeckSF", "OpenPorchSF", "3SsnPorch", "ScreenPorch"]

# RANDOM FOREST

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

import xgboost as xgb

target = train['SalePrice'] # target
target = np.log(target)

y = target
X = train[vars_to_keep]

# model = RandomForestRegressor(bootstrap = False, max_depth=None, max_features='sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 500, random_state = 42)

model = xgb.XGBRegressor(tree_method="hist", max_depth=2)
model.fit(X, y)
score = cross_val_score(model, X, y, cv=10)
print(score.mean())



0.8813061342125827


# FINAL OUTPUT

In [10]:
price = model.predict(test[vars_to_keep])
price = np.exp(price)
submission = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": price
})

submission.to_csv("submission.csv", index=False)
submission.sample(10)

,Id,SalePrice
638,2099,45095.660156
1300,2761,144813.578125
276,1737,358123.218750
729,2190,82884.148438
1423,2884,197607.187500
788,2249,118981.421875
19,1480,413671.281250
403,1864,397607.968750
1413,2874,140301.937500
75,1536,115156.898438
